In [ ]:
%matplotlib qt

In [ ]:
import matplotlib.pyplot as plt
import numpy
import math
import sympy

In [ ]:
class Quaternion(object):
    def __init__(self,a,b,c,d):
        self.a = a
        self.b = b
        self.c = c
        self.d = d

    def __mul__(self,other):
        a = self.a*other.a - self.b*other.b - self.c*other.c - self.d*other.d
        b = self.a*other.b + self.b*other.a + self.c*other.d - self.d*other.c
        c = self.a*other.c + self.c*other.a + self.d*other.b - self.b*other.d
        d = self.a*other.d + self.d*other.a + self.b*other.c - self.c*other.b
        quat = Quaternion(a,b,c,d)
        return quat
    
    def __add__(self,other):
        a = self.a+other.a
        b = self.b+other.b
        c = self.c+other.c
        d = self.d+other.d
        q = Quaternion(a,b,c,d)
        return q
    
    def __neg__(self):
        return Quaternion(-self.a,-self.b,-self.c,-self.d)
    
    def __sub__(self,other):
        return (self + (-other))
    
    def __str__(self):
        s = '{}+{}i+{}j+{}k'.format(self.a,self.b,self.c,self.d)
        return s
    
    def __repr__(self):
        return str(self)
    
    def conj(self):
        return self.conjugate()

    def conjugate(self):
        return Quaternion(self.a,-self.b,-self.c,-self.d)

    def real(self):
        return self.a

    def imaginary(self):
        return self.b,self.c,self.d

    @staticmethod
    def from_axis_angle(v,theta,symbolic = False):
        v = v.flatten()
        l = (v.dot(v))**.5
        v = (v/l).tolist()
        if symbolic:
            import sympy
            a = sympy.cos(theta/2)
            s = sympy.sin(theta/2)
        else:
            a = math.cos(theta/2)
            s = math.sin(theta/2)
        b = s*v[0]
        c = s*v[1]
        d = s*v[2]
        q = Quaternion(a,b,c,d)
        return q
    
    def to_rot(self):
        v1 = self*Quaternion(0,1,0,0)*self.conjugate()
        v2 = self*Quaternion(0,0,1,0)*self.conjugate()
        v3 = self*Quaternion(0,0,0,1)*self.conjugate()

        v1 = numpy.array([v1.simplify().imaginary()]).T
        v2 = numpy.array([v2.simplify().imaginary()]).T
        v3 = numpy.array([v3.simplify().imaginary()]).T

        R = numpy.hstack([v1,v2,v3])

        return R
    
    def simplify(self):
        a = self.a.simplify()
        b = self.b.simplify()
        c = self.c.simplify()
        d = self.d.simplify()
        return Quaternion(a,b,c,d)

    def subs(self,dict1):
        try:
            a = self.a.subs(dict1)
        except AttributeError:
            a = self.a
        try:
            b = self.b.subs(dict1)
        except AttributeError:
            b = self.b
        try:
            c = self.c.subs(dict1)
        except AttributeError:
            c = self.c
        try:
            d = self.d.subs(dict1)
        except AttributeError:
            d = self.d                        

        return Quaternion(a,b,c,d)

In [ ]:
v0 = [0,0,0]
v1 = [1,0,0]
v2 = [1,1,0]
v3 = [-1,1,0]
v4 = [0,-1,0]

v0 = numpy.array([v0]).T
v1 = numpy.array([v1]).T
v2 = numpy.array([v2]).T
v3 = numpy.array([v3]).T
v4 = numpy.array([v4]).T

v = numpy.hstack([v0,v1,v2,v3,v4])

plt.plot(v[0,(0,1,0,2,0,3,0,4,3,2,1)],v[1,(0,1,0,2,0,3,0,4,3,2,1)])

In [ ]:
a1, a2, a3,a4 = sympy.symbols('a1,a2,a3,a4')

In [ ]:
r1 = Quaternion.from_axis_angle(v1,a1,symbolic=True)
r2 = Quaternion.from_axis_angle(v2,a2,symbolic=True)
r3 = Quaternion.from_axis_angle(v3,a3,symbolic=True)
r4 = Quaternion.from_axis_angle(v4,a4,symbolic=True)

In [ ]:
r1

In [ ]:
r2

In [ ]:
p1 = Quaternion(0,*v1.flatten().tolist())
p2 = Quaternion(0,*v2.flatten().tolist())
p3 = Quaternion(0,*v3.flatten().tolist())
p4 = Quaternion(0,*v4.flatten().tolist())

In [ ]:
p1

In [ ]:
p4

In [ ]:
p3p = r2*p3*r2.conj()
p4p = r2*r3*p4*r3.conj()*r2.conj()
p1p = r2*r3*r4*p1*r4.conj()*r3.conj()*r2.conj()
p2p = r2*r3*r4*r1*p2*r1.conj()*r4.conj()*r3.conj()*r2.conj()

In [ ]:
p3pv = numpy.array([p3p.imaginary()]).T
p4pv = numpy.array([p4p.imaginary()]).T
p1pv = numpy.array([p1p.imaginary()]).T
p2pv = numpy.array([p2p.imaginary()]).T

x = [30*math.pi/180,30*math.pi/180,30*math.pi/180,30*math.pi/180]

alpha1,alpha2,alpha3,alpha4 = x

subs_dict = {a1:alpha1,a2:alpha2,a3:alpha3,a4:alpha4}
    
p_p = numpy.hstack([v0,v1,v2,p3pv,p4pv,p1pv,p2pv])
p_p = sympy.Matrix(p_p)
p_p = p_p.subs(subs_dict)
p_p = numpy.array(p_p,dtype = float)

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.plot(*p_p[:,(0,1,0,2,0,3,0,4,0,5,0,6,5,4,3,2,1)])
plt.axis('equal')
plt.show()